# Downgrade tensorflow to V1

In [1]:
! pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5 MB 7.8 kB/s 
     |████████████████████████████████| 503 kB 33.7 MB/s 
     |████████████████████████████████| 3.8 MB 33.1 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=137da57f533ffd849b19f888aa23c9c9345b6dfa639b9e642ececa07431d5463
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succe

# Imports

In [2]:
from __future__ import division, print_function, absolute_import
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn import preprocessing
import os
import time

import tensorflow as tf

from xgboost import XGBClassifier
import datetime


# Loading Functions

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo,encoding='bytes')
    return dict


def reconstructlabel(label):
    length = len(label)
    newlabel = []
    for index in range(length):
        if label[index] == 0:
            newlabel.append([0, 1])
        else:
            newlabel.append([1, 0])
    return newlabel

# Download Datasets

## Amazon Opinion Fraud (Used in Paper)

In [ ]:
# Amazon
! gdown --id 17leD629H6QsruNiNKYMw1k-yhW3sbeKM

Downloading...
From: https://drive.google.com/uc?id=17leD629H6QsruNiNKYMw1k-yhW3sbeKM
To: /content/dataset.p
100% 31.2M/31.2M [00:00<00:00, 145MB/s]


In [ ]:
am_data = unpickle('dataset.p')

body = am_data[1:,:]
arr_am_X = body[:,1:]
arr_am_Y = body[:,0]


## Ethereum Fraud

In [ ]:
# Ethereum
# https://www.kaggle.com/vagifa/ethereum-frauddetection-dataset
! gdown --id 1WF37JnENj_Y3Mu661DYFotCiGAo2XB-7

Downloading...
From: https://drive.google.com/uc?id=1WF37JnENj_Y3Mu661DYFotCiGAo2XB-7
To: /content/etherum_transaction_dataset.csv
100% 2.88M/2.88M [00:00<00:00, 91.9MB/s]


In [ ]:
df_eth = pd.read_csv('etherum_transaction_dataset.csv')

df_eth.head()

,Unnamed: 0,Index,Address,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,min value received,max value received,avg val received,min val sent,max val sent,avg val sent,min value sent to contract,max val sent to contract,avg value sent to contract,total transactions (including tnx to create contract,total Ether sent,total ether received,total ether sent contracts,total ether balance,Total ERC20 tnxs,ERC20 total Ether received,ERC20 total ether sent,ERC20 total Ether sent contract,ERC20 uniq sent addr,ERC20 uniq rec addr,ERC20 uniq sent addr.1,ERC20 uniq rec contract addr,ERC20 avg time between sent tnx,ERC20 avg time between rec tnx,ERC20 avg time between rec 2 tnx,ERC20 avg time between contract tnx,ERC20 min val rec,ERC20 max val rec,ERC20 avg val rec,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name,ERC20 most sent token type,ERC20_most_rec_token_type
0,0,1,0x00009277775ac7d0d59eaad8fee3d10ac6c805e8,0,844.26,1093.71,704785.63,721,89,0,40,118,0.000000,45.806785,6.589513,0.00,31.220000,1.200681,0.0,0.0,0.0,810,865.691093,586.466675,0.0,-279.224419,265.0,3.558854e+07,3.560317e+07,0.0,30.0,54.0,0.0,58.0,0.0,0.0,0.0,0.0,0.0,1.500000e+07,265586.147600,0.000000,1.683100e+07,271779.920000,0.0,0.0,0.0,39.0,57.0,Cofoundit,Numeraire
1,1,2,0x0002b44ddb1476db43c868bd494422ee4c136fed,0,12709.07,2958.44,1218216.73,94,8,0,5,14,0.000000,2.613269,0.385685,0.00,1.800000,0.032844,0.0,0.0,0.0,102,3.087297,3.085478,0.0,-0.001819,8.0,4.034283e+02,2.260809e+00,0.0,1.0,5.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,3.650000e+02,57.632615,2.260809,2.260809e+00,2.260809,0.0,0.0,0.0,1.0,7.0,Livepeer Token,Livepeer Token
2,2,3,0x0002bda54cb772d040f779e88eb453cac0daa244,0,246194.54,2434.02,516729.30,2,10,0,10,2,0.113119,1.165453,0.358906,0.05,3.538616,1.794308,0.0,0.0,0.0,12,3.588616,3.589057,0.0,0.000441,8.0,5.215121e+02,0.000000e+00,0.0,0.0,7.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,4.428198e+02,65.189009,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,8.0,None,XENON
3,3,4,0x00038e6ba2fd5c09aedb96697c8d7b8fa6632e5e,0,10219.60,15785.09,397555.90,25,9,0,7,13,0.000000,500.000000,99.488840,0.00,450.000000,70.001834,0.0,0.0,0.0,34,1750.045862,895.399559,0.0,-854.646303,14.0,1.711105e+04,1.141223e+04,0.0,2.0,11.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,1.141223e+04,1555.550174,100.000000,9.029231e+03,3804.076893,0.0,0.0,0.0,1.0,11.0,Raiden,XENON
4,4,5,0x00062d1dd1afb6fb02540ddad9cdebfe568e0d89,0,36.61,10707.77,382472.42,4598,20,1,7,19,0.000000,12.802411,2.671095,0.00,9.000000,0.022688,0.0,0.0,0.0,4619,104.318883,53.421896,0.0,-50.896986,42.0,1.628297e+05,1.235399e+05,0.0,4.0,23.0,0.0,27.0,0.0,0.0,0.0,0.0,0.0,9.000000e+04,4934.232147,0.000000,4.500000e+04,13726.659220,0.0,0.0,0.0,6.0,27.0,StatusNetwork,EOS


In [ ]:
df_eth.describe()

,Unnamed: 0,Index,FLAG,Avg min between sent tnx,Avg min between received tnx,Time Diff between first and last (Mins),Sent tnx,Received Tnx,Number of Created Contracts,Unique Received From Addresses,Unique Sent To Addresses,min value received,max value received,avg val received,min val sent,max val sent,avg val sent,min value sent to contract,max val sent to contract,avg value sent to contract,total transactions (including tnx to create contract,total Ether sent,total ether received,total ether sent contracts,total ether balance,Total ERC20 tnxs,ERC20 total Ether received,ERC20 total ether sent,ERC20 total Ether sent contract,ERC20 uniq sent addr,ERC20 uniq rec addr,ERC20 uniq sent addr.1,ERC20 uniq rec contract addr,ERC20 avg time between sent tnx,ERC20 avg time between rec tnx,ERC20 avg time between rec 2 tnx,ERC20 avg time between contract tnx,ERC20 min val rec,ERC20 max val rec,ERC20 avg val rec,ERC20 min val sent,ERC20 max val sent,ERC20 avg val sent,ERC20 min val sent contract,ERC20 max val sent contract,ERC20 avg val sent contract,ERC20 uniq sent token name,ERC20 uniq rec token name
count,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9.841000e+03,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9841.000000,9.841000e+03,9.841000e+03,9841.000000,9.841000e+03,9012.000000,9.012000e+03,9.012000e+03,9012.000000,9012.000000,9012.000000,9012.000000,9012.000000,9012.0,9012.0,9012.0,9012.0,9012.000000,9.012000e+03,9.012000e+03,9.012000e+03,9.012000e+03,9.012000e+03,9012.0,9012.0,9012.0,9012.000000,9012.000000
mean,4920.000000,1815.049893,0.221421,5086.878721,8004.851184,2.183333e+05,115.931714,163.700945,3.729702,30.360939,25.840159,43.845153,523.152481,100.711721,4.800090,314.617297,44.755731,0.000003,0.000008,0.000005,283.362362,1.016092e+04,1.163832e+04,0.000008,1.477395e+03,36.255659,1.296207e+08,1.386849e+07,110.939207,5.638038,7.598535,0.003440,4.901909,0.0,0.0,0.0,0.0,485.614688,1.252524e+08,4.346203e+06,1.174126e+04,1.303594e+07,6.318389e+06,0.0,0.0,0.0,1.384931,4.826676
std,2840.996333,1222.621830,0.415224,21486.549974,23081.714801,3.229379e+05,757.226361,940.836550,141.445583,298.621112,263.820410,325.929139,13008.821539,2885.002236,138.609682,6629.212643,239.080215,0.000225,0.000516,0.000323,1352.404013,3.583227e+05,3.642048e+05,0.000516,2.424254e+05,447.528908,1.053858e+10,1.180390e+09,6128.634953,105.252500,81.818470,0.065698,17.246576,0.0,0.0,0.0,0.0,16883.278712,1.053741e+10,2.141192e+08,1.053567e+06,1.179905e+09,5.914764e+08,0.0,0.0,0.0,6.735121,16.678607
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,-1.560535e+07,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000
25%,2460.000000,821.000000,0.000000,0.000000,0.000000,3.169300e+02,1.000000,1.000000,0.000000,1.000000,1.000000,0.001000,1.000000,0.426905,0.000000,0.164577,0.086184,0.000000,0.000000,0.000000,4.000000,2.262059e-01,2.670424e+00,0.000000,6.214900e-04,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000
50%,4920.000000,1641.000000,0.000000,17.340000,509.770000,4.663703e+04,3.000000,4.000000,0.000000,2.000000,2.000000,0.095856,6.000000,1.729730,0.049126,4.999380,1.606000,0.000000,0.000000,0.000000,8.000000,1.248680e+01,3.052963e+01,0.000000,1.722000e-03,1.000000,1.000000e-12,0.000000e+00,0.000000,0.000000,1.000000,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.000000,1.000000
75%,7

# Data Pre-processing

## Amazon Opinion Fraud

In [ ]:
arr_am_X_norm=preprocessing.scale(arr_am_X)

In [ ]:
# train-val split

x_train_am, x_val_am, y_train_am, y_val_am = train_test_split(arr_am_X_norm, arr_am_Y.astype(int), test_size=0.33, random_state=42)

In [ ]:
y_train_am_rl = np.array(reconstructlabel(y_train_am))
y_val_am_rl = np.array(reconstructlabel(y_val_am))

## Ethereum Fraud

In [ ]:
# Get X variables only + drop last 2 categorical variables - poor model performance with one-hot

df_eth_X = df_eth.drop(['Unnamed: 0', 'Index', 'Address', 'FLAG', df_eth.columns[-2], df_eth.columns[-1]], axis = 1)

# Fix NAN
df_eth_X = df_eth_X.fillna(0)

# Scale continuous variables
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(df_eth_X.values)
#df_eth_X = pd.DataFrame(x_scaled, columns = df_eth_X.columns)
#df_eth_X.head()
df_eth_X_norm=preprocessing.scale(df_eth_X)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [ ]:
x_train_eth, x_val_eth, y_train_eth, y_val_eth = train_test_split(df_eth_X_norm, df_eth['FLAG'], test_size=0.33, random_state=42)

In [ ]:
y_train_eth_rl = np.array(reconstructlabel(y_train_eth.to_list()))
y_val_eth_rl = np.array(reconstructlabel(y_val_eth.to_list()))

# Model Parameters

In [ ]:
# Parameter Settings
DEPTH   = 3                 # Depth of a tree
N_LEAF  = 2 ** (DEPTH + 1)  # Number of leaf node 
N_LABEL = 2                # Number of classes
N_TREE  = 5                 # Number of trees (ensemble)
N_BATCH = 50             # Number of data points per mini-batch

# Model Functions

## Utility Functions

In [ ]:
# ======================functions================================ #

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


# constructing neural random forest
def model(input_layer, w_t_e, w_d_e, w_l_e, p_keep_hidden):
    assert (len(w_t_e) == len(w_d_e))
    assert (len(w_t_e) == len(w_l_e))
    decision_p_e = []
    leaf_p_e = []
    for w_t, w_d, w_l in zip(w_t_e, w_d_e, w_l_e):
        tree_layer = tf.nn.relu(tf.matmul(input_layer, w_t))
        tree_layer = tf.nn.dropout(tree_layer, p_keep_hidden)

        decision_p = tf.nn.sigmoid(tf.matmul(tree_layer, w_d)) 
        leaf_p = tf.nn.softmax(w_l) 

        decision_p_e.append(decision_p)
        leaf_p_e.append(leaf_p)
    return decision_p_e, leaf_p_e

def neural_random_forest(decision_p_e, leaf_p_e, n_depth, n_leaf, n_batch):
    flat_decision_p_e = []

    for decision_p in decision_p_e:
        decision_p_comp = tf.subtract(tf.ones_like(decision_p), decision_p)
        decision_p_pack = tf.stack([decision_p, decision_p_comp])
        flat_decision_p = tf.reshape(decision_p_pack, [-1])
        flat_decision_p_e.append(flat_decision_p)

    batch_0_indices = \
        tf.tile(tf.expand_dims(tf.range(0, n_batch * n_leaf, n_leaf), 1), [1, n_leaf])

    # The routing probability computation
    in_repeat = int(n_leaf / 2)
    out_repeat = n_batch

    batch_complement_indices = \
        np.array([[0] * in_repeat, [n_batch * n_leaf] * in_repeat]
                 * out_repeat).reshape(n_batch, n_leaf)

    mu_e = []
    # iterate over each tree
    for i, flat_decision_p in enumerate(flat_decision_p_e):
        mu = tf.gather(flat_decision_p, tf.add(batch_0_indices, batch_complement_indices))
        mu_e.append(mu)

    # from the second layer to the last layer, we make the decision nodes
    for d in range(1, n_depth + 1):
        indices = tf.range(2 ** d, 2 ** (d + 1)) - 1
        tile_indices = tf.reshape(tf.tile(tf.expand_dims(indices, 1),
                                          [1, 2 ** (n_depth - d + 1)]), [1, -1])
        batch_indices = tf.add(batch_0_indices, tf.tile(tile_indices, [n_batch, 1]))

        in_repeat = int(in_repeat / 2)
        out_repeat = int(out_repeat * 2)

        batch_complement_indices = \
            np.array([[0] * in_repeat, [n_batch * n_leaf] * in_repeat]
                     * out_repeat).reshape(n_batch, n_leaf)

        mu_e_update = []
        for mu, flat_decision_p in zip(mu_e, flat_decision_p_e):
            mu = tf.multiply(mu, tf.gather(flat_decision_p, tf.add(batch_indices, batch_complement_indices)))
            mu_e_update.append(mu)

        mu_e = mu_e_update

    return mu_e


def probability_y_x(mu_e, leaf_p_e):
    py_x_e = []
    py_x_leaf_e = []
    for mu, leaf_p in zip(mu_e, leaf_p_e):
        # average all the leaf p
        py_x_tree = tf.reduce_sum(
            tf.multiply(tf.tile(tf.expand_dims(mu, 2), [1, 1, N_LABEL]),
                        tf.tile(tf.expand_dims(leaf_p, 0), [N_BATCH, 1, 1])), 1)
        a_tree = tf.multiply(tf.tile(tf.expand_dims(mu, 2), [1, 1, N_LABEL]),
                             tf.tile(tf.expand_dims(leaf_p, 0), [N_BATCH, 1, 1]))
        py_x_e.append(py_x_tree)
        py_x_leaf_e.append(a_tree)

    py_x_e = tf.stack(py_x_e)
    py_x = tf.reduce_sum(py_x_e, 0)
    final = tf.nn.softmax(py_x)
    # py_x_leaf_e = tf.stack(py_x_leaf_e)
    return final


def init_prob_weights(shape, minval=-5, maxval=5):
    return tf.Variable(tf.random_uniform(shape, minval, maxval))



## Actual Models

### Autoencoder + Decision Forest

Taken from the paper's author and modified (ensure that we are using the same loss function, optimizer network structure and other hyperparameters:

https://github.com/dongmanqing/Opinion-Fraud-Detection-via-Neural-Autoencoder-Decision-Forest/blob/master/main.py

Uses sigmoid activation for autoencoder

In [ ]:
def run_model(N_feature, trX, teX, trY, teY, num_epoch, early_stop):


    # Input X, output Y
    x = tf.placeholder("float", [N_BATCH, N_feature])
    y = tf.placeholder("float", [N_BATCH, N_LABEL])

    p_keep_conv = tf.placeholder("float")
    p_keep_hidden = tf.placeholder("float")

    # ====================Autoencoder========================================#
    num_hidden_1 = 64  
    num_hidden_2 = 96  
    # num_hidden_3 = 64
    # num_hidden_4 = 96
    num_input = N_feature  

    weights = {
        'encoder_h1': tf.Variable(tf.truncated_normal([num_input, num_hidden_1], stddev=0.1)),
        'encoder_h2': tf.Variable(tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=0.1)),
        # 'encoder_h3': tf.Variable(tf.truncated_normal([num_hidden_2, num_hidden_3], stddev=0.1)),
        # 'encoder_h4': tf.Variable(tf.truncated_normal([num_hidden_3, num_hidden_4], stddev=0.1)),
        # 'decoder_h1': tf.Variable(tf.truncated_normal([num_hidden_4, num_hidden_3], stddev=0.1)),
        # 'decoder_h2': tf.Variable(tf.truncated_normal([num_hidden_3, num_hidden_2], stddev=0.1)),
        'decoder_h1': tf.Variable(tf.truncated_normal([num_hidden_2, num_hidden_1], stddev=0.1)),
        'decoder_h2': tf.Variable(tf.truncated_normal([num_hidden_1, num_input], stddev=0.1))
    }

    biases = {
        'encoder_b1': tf.Variable(tf.constant(0.1, shape=[num_hidden_1])),
        'encoder_b2': tf.Variable(tf.constant(0.1, shape=[num_hidden_2])),
        # 'encoder_b3': tf.Variable(tf.constant(0.1, shape=[num_hidden_3])),
        # 'encoder_b4': tf.Variable(tf.constant(0.1, shape=[num_hidden_4])),
        # 'decoder_b1': tf.Variable(tf.constant(0.1, shape=[num_hidden_3])),
        # 'decoder_b2': tf.Variable(tf.constant(0.1, shape=[num_hidden_2])),
        'decoder_b1': tf.Variable(tf.constant(0.1, shape=[num_hidden_1])),
        'decoder_b2': tf.Variable(tf.constant(0.1, shape=[num_input]))
    }


    # Building the encoder
    def encoder(x):
        # Encoder Hidden layer with sigmoid activation #1
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                      biases['encoder_b1']))
        # Encoder Hidden layer with sigmoid activation #2
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                      biases['encoder_b2']))
        # layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
        #                                biases['encoder_b3']))
        # layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
        #                                biases['encoder_b4']))

        return layer_2


    # Building the decoder
    def decoder(x):
        # Decoder Hidden layer with sigmoid activation #1
        layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                      biases['decoder_b1']))
        # Decoder Hidden layer with sigmoid activation #2
        layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                      biases['decoder_b2']))

        # layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
        #                                biases['decoder_b3']))
        # layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
        #                                biases['decoder_b4']))
        return layer_2

    # Construct model
    encoder_op = encoder(x)
    decoder_op = decoder(encoder_op)

    # Prediction
    y_pred = decoder_op
    # Targets (Labels) are the input data.
    y_true = x

    # ====================Neural decision forest=================================#
    num_out = 96
    num_output = 128
    N_nodes = 256

    W_hidden = weight_variable([num_out, num_output])
    b_hidden = weight_variable([num_output])
    tree_input = tf.nn.relu(tf.matmul(encoder_op, W_hidden) + b_hidden)
    tree_input = tf.nn.dropout(tree_input, p_keep_conv)

    w_t_ensemble = []
    w_d_ensemble = []
    w_l_ensemble = []
    for i in range(N_TREE):
        w_t_ensemble.append(weight_variable([num_output, N_nodes])) 
        w_d_ensemble.append(init_prob_weights([N_nodes, N_LEAF], -1, 1)) 
        w_l_ensemble.append(
            init_prob_weights([N_LEAF, N_LABEL], -2, 2))  

    decision_p_e, leaf_p_e = model(tree_input, w_t_ensemble, w_d_ensemble, w_l_ensemble, p_keep_hidden)
    mu_e = neural_random_forest(decision_p_e, leaf_p_e, DEPTH, N_LEAF, N_BATCH)
    py_x = probability_y_x(mu_e, leaf_p_e) 

    # loss function
    cost = tf.add(tf.reduce_mean(-tf.multiply(tf.log(py_x), y)),tf.reduce_mean(tf.pow(y_true - y_pred, 2)))
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    predict = (tf.argmax(py_x, 1), py_x)

    # =============== training ========================== #
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    largest_auc = 0.0
    counter = 0
    for i in range(num_epoch):
        # One epoch
        for start, end in zip(range(0, len(trX), N_BATCH), range(N_BATCH, len(trX), N_BATCH)):
            sess.run(train_step, feed_dict={x: trX[start:end], y: trY[start:end],
                                            p_keep_conv: 0.8, p_keep_hidden: 0.5})
        results = []
        prob = []
        true = []
        for start, end in zip(range(0, len(teX), N_BATCH), range(N_BATCH, len(teX), N_BATCH)):
            predicted = sess.run(predict, feed_dict={x: teX[start:end], p_keep_conv:1.0, p_keep_hidden: 1.0})
            results.extend(np.argmax(teY[start:end], axis=1) == predicted[0])

            prob.extend(predicted[1][:,1])
            true.extend(np.argmax(teY[start:end], axis=1))

        fpr, tpr, thresholds = metrics.roc_curve(true, prob, pos_label=1)
        auc_score = metrics.auc(fpr, tpr)
        print('Epoch: %d, Test Accuracy: %f, AUC: %f' % (i + 1, np.mean(results), auc_score))

        if (auc_score > largest_auc):
            largest_auc = auc_score
        else:
            counter += 1

        if (counter >= early_stop):
            print('Early stopping...AUC has not improved for %d rounds' % (counter))
            print('Best AUC: %f', (largest_auc))
            break





### Autoencoder ReLU Modification

In [ ]:
def run_model_relu(N_feature, trX, teX, trY, teY, num_epoch, early_stop):


    # Input X, output Y
    x = tf.placeholder("float", [N_BATCH, N_feature])
    y = tf.placeholder("float", [N_BATCH, N_LABEL])

    p_keep_conv = tf.placeholder("float")
    p_keep_hidden = tf.placeholder("float")

    # ====================Autoencoder========================================#
    num_hidden_1 = 64  
    num_hidden_2 = 96  
    # num_hidden_3 = 64
    # num_hidden_4 = 96
    num_input = N_feature  

    weights = {
        'encoder_h1': tf.Variable(tf.truncated_normal([num_input, num_hidden_1], stddev=0.1)),
        'encoder_h2': tf.Variable(tf.truncated_normal([num_hidden_1, num_hidden_2], stddev=0.1)),
        # 'encoder_h3': tf.Variable(tf.truncated_normal([num_hidden_2, num_hidden_3], stddev=0.1)),
        # 'encoder_h4': tf.Variable(tf.truncated_normal([num_hidden_3, num_hidden_4], stddev=0.1)),
        # 'decoder_h1': tf.Variable(tf.truncated_normal([num_hidden_4, num_hidden_3], stddev=0.1)),
        # 'decoder_h2': tf.Variable(tf.truncated_normal([num_hidden_3, num_hidden_2], stddev=0.1)),
        'decoder_h1': tf.Variable(tf.truncated_normal([num_hidden_2, num_hidden_1], stddev=0.1)),
        'decoder_h2': tf.Variable(tf.truncated_normal([num_hidden_1, num_input], stddev=0.1))
    }

    biases = {
        'encoder_b1': tf.Variable(tf.constant(0.1, shape=[num_hidden_1])),
        'encoder_b2': tf.Variable(tf.constant(0.1, shape=[num_hidden_2])),
        # 'encoder_b3': tf.Variable(tf.constant(0.1, shape=[num_hidden_3])),
        # 'encoder_b4': tf.Variable(tf.constant(0.1, shape=[num_hidden_4])),
        # 'decoder_b1': tf.Variable(tf.constant(0.1, shape=[num_hidden_3])),
        # 'decoder_b2': tf.Variable(tf.constant(0.1, shape=[num_hidden_2])),
        'decoder_b1': tf.Variable(tf.constant(0.1, shape=[num_hidden_1])),
        'decoder_b2': tf.Variable(tf.constant(0.1, shape=[num_input]))
    }


    # Building the encoder
    def encoder(x):
        # Encoder Hidden layer with sigmoid activation #1
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['encoder_h1']),
                                      biases['encoder_b1']))
        # Encoder Hidden layer with sigmoid activation #2
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                      biases['encoder_b2']))
        # layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
        #                                biases['encoder_b3']))
        # layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
        #                                biases['encoder_b4']))

        return layer_2


    # Building the decoder
    def decoder(x):
        # Decoder Hidden layer with sigmoid activation #1
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['decoder_h1']),
                                      biases['decoder_b1']))
        # Decoder Hidden layer with sigmoid activation #2
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                      biases['decoder_b2']))

        # layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
        #                                biases['decoder_b3']))
        # layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
        #                                biases['decoder_b4']))
        return layer_2

    # Construct model
    encoder_op = encoder(x)
    decoder_op = decoder(encoder_op)

    # Prediction
    y_pred = decoder_op
    # Targets (Labels) are the input data.
    y_true = x

    # ====================Neural decision forest=================================#
    num_out = 96
    num_output = 128
    N_nodes = 256

    W_hidden = weight_variable([num_out, num_output])
    b_hidden = weight_variable([num_output])
    tree_input = tf.nn.relu(tf.matmul(encoder_op, W_hidden) + b_hidden)
    tree_input = tf.nn.dropout(tree_input, p_keep_conv)

    w_t_ensemble = []
    w_d_ensemble = []
    w_l_ensemble = []
    for i in range(N_TREE):
        w_t_ensemble.append(weight_variable([num_output, N_nodes])) 
        w_d_ensemble.append(init_prob_weights([N_nodes, N_LEAF], -1, 1)) 
        w_l_ensemble.append(
            init_prob_weights([N_LEAF, N_LABEL], -2, 2))  

    decision_p_e, leaf_p_e = model(tree_input, w_t_ensemble, w_d_ensemble, w_l_ensemble, p_keep_hidden)
    mu_e = neural_random_forest(decision_p_e, leaf_p_e, DEPTH, N_LEAF, N_BATCH)
    py_x = probability_y_x(mu_e, leaf_p_e) 

    # loss function
    cost = tf.add(tf.reduce_mean(-tf.multiply(tf.log(py_x), y)),tf.reduce_mean(tf.pow(y_true - y_pred, 2)))
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    predict = (tf.argmax(py_x, 1), py_x)

    # =============== training ========================== #
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    largest_auc = 0.0
    counter = 0
    for i in range(num_epoch):
        # One epoch
        for start, end in zip(range(0, len(trX), N_BATCH), range(N_BATCH, len(trX), N_BATCH)):
            sess.run(train_step, feed_dict={x: trX[start:end], y: trY[start:end],
                                            p_keep_conv: 0.8, p_keep_hidden: 0.5})
        results = []
        prob = []
        true = []
        for start, end in zip(range(0, len(teX), N_BATCH), range(N_BATCH, len(teX), N_BATCH)):
            predicted = sess.run(predict, feed_dict={x: teX[start:end], p_keep_conv:1.0, p_keep_hidden: 1.0})
            results.extend(np.argmax(teY[start:end], axis=1) == predicted[0])

            prob.extend(predicted[1][:,1])
            true.extend(np.argmax(teY[start:end], axis=1))

        fpr, tpr, thresholds = metrics.roc_curve(true, prob, pos_label=1)
        auc_score = metrics.auc(fpr, tpr)
        print('Epoch: %d, Test Accuracy: %f, AUC: %f' % (i + 1, np.mean(results), auc_score))

        if (auc_score > largest_auc):
            largest_auc = auc_score
        else:
            counter += 1

        if (counter >= early_stop):
            print('Early stopping...AUC has not improved for %d rounds' % (counter))
            print('Best AUC: %f', (largest_auc))
            break





# Fit Models

## Amazon Opinion Fraud

### Autoencoder + Neural DF

#### Original (sigmoid)

In [ ]:
start = datetime.datetime.now()
run_model(len(x_train_am[0]), x_train_am, x_val_am, y_train_am_rl, y_val_am_rl, 1000, 20)
time_taken = datetime.datetime.now() - start
print(time_taken)

Epoch: 1, Test Accuracy: 0.583462, AUC: 0.545567
Epoch: 2, Test Accuracy: 0.583462, AUC: 0.650805
Epoch: 3, Test Accuracy: 0.630385, AUC: 0.710681
Epoch: 4, Test Accuracy: 0.708077, AUC: 0.742312
Epoch: 5, Test Accuracy: 0.703077, AUC: 0.753090
Epoch: 6, Test Accuracy: 0.705000, AUC: 0.758091
Epoch: 7, Test Accuracy: 0.720000, AUC: 0.763273
Epoch: 8, Test Accuracy: 0.718462, AUC: 0.767938
Epoch: 9, Test Accuracy: 0.727692, AUC: 0.772420
Epoch: 10, Test Accuracy: 0.731154, AUC: 0.775746
Epoch: 11, Test Accuracy: 0.733077, AUC: 0.777267
Epoch: 12, Test Accuracy: 0.738077, AUC: 0.779760
Epoch: 13, Test Accuracy: 0.741923, AUC: 0.777944
Epoch: 14, Test Accuracy: 0.748077, AUC: 0.782222
Epoch: 15, Test Accuracy: 0.758077, AUC: 0.790767
Epoch: 16, Test Accuracy: 0.748846, AUC: 0.795866
Epoch: 17, Test Accuracy: 0.758462, AUC: 0.799020
Epoch: 18, Test Accuracy: 0.753846, AUC: 0.804213
Epoch: 19, Test Accuracy: 0.758846, AUC: 0.808781
Epoch: 20, Test Accuracy: 0.744615, AUC: 0.807920
Epoch: 21

#### ReLU version

In [ ]:
start = datetime.datetime.now()
run_model_relu(len(x_train_am[0]), x_train_am, x_val_am, y_train_am_rl, y_val_am_rl, 1000, 20)
time_taken = datetime.datetime.now() - start
print(time_taken)

Epoch: 1, Test Accuracy: 0.623077, AUC: 0.693669
Epoch: 2, Test Accuracy: 0.725000, AUC: 0.774613
Epoch: 3, Test Accuracy: 0.754615, AUC: 0.815221
Epoch: 4, Test Accuracy: 0.780000, AUC: 0.844526
Epoch: 5, Test Accuracy: 0.823846, AUC: 0.877797
Epoch: 6, Test Accuracy: 0.833077, AUC: 0.894082
Epoch: 7, Test Accuracy: 0.848846, AUC: 0.915288
Epoch: 8, Test Accuracy: 0.854615, AUC: 0.928205
Epoch: 9, Test Accuracy: 0.874231, AUC: 0.946706
Epoch: 10, Test Accuracy: 0.888077, AUC: 0.949047
Epoch: 11, Test Accuracy: 0.880385, AUC: 0.956342
Epoch: 12, Test Accuracy: 0.896538, AUC: 0.962600
Epoch: 13, Test Accuracy: 0.909231, AUC: 0.967635
Epoch: 14, Test Accuracy: 0.903846, AUC: 0.961963
Epoch: 15, Test Accuracy: 0.922692, AUC: 0.973405
Epoch: 16, Test Accuracy: 0.932692, AUC: 0.975755
Epoch: 17, Test Accuracy: 0.916923, AUC: 0.970754
Epoch: 18, Test Accuracy: 0.932692, AUC: 0.975758
Epoch: 19, Test Accuracy: 0.941154, AUC: 0.980916
Epoch: 20, Test Accuracy: 0.935385, AUC: 0.975057
Epoch: 21

### XGBoost

In [ ]:
start = datetime.datetime.now()

xgbmodel_am = XGBClassifier(objective='binary:logistic',  n_estimators = 1000, booster = 'gbtree', eval_metric = ['auc'])
xgbmodel_am.fit(x_train_am, y_train_am, eval_set = [(x_val_am, y_val_am)], early_stopping_rounds = 10)
time_taken = datetime.datetime.now() - start
print(time_taken)

[0]	validation_0-auc:0.771731
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.807045
[2]	validation_0-auc:0.813855
[3]	validation_0-auc:0.822107
[4]	validation_0-auc:0.836414
[5]	validation_0-auc:0.836386
[6]	validation_0-auc:0.849798
[7]	validation_0-auc:0.850475
[8]	validation_0-auc:0.852863
[9]	validation_0-auc:0.859037
[10]	validation_0-auc:0.865218
[11]	validation_0-auc:0.868144
[12]	validation_0-auc:0.875424
[13]	validation_0-auc:0.879622
[14]	validation_0-auc:0.88507
[15]	validation_0-auc:0.887647
[16]	validation_0-auc:0.890419
[17]	validation_0-auc:0.893082
[18]	validation_0-auc:0.89435
[19]	validation_0-auc:0.897332
[20]	validation_0-auc:0.900616
[21]	validation_0-auc:0.901814
[22]	validation_0-auc:0.904788
[23]	validation_0-auc:0.906233
[24]	validation_0-auc:0.907148
[25]	validation_0-auc:0.908366
[26]	validation_0-auc:0.910039
[27]	validation_0-auc:0.912891
[28]	validation_0-auc:0.915357
[29]	validation_0-auc:0.917449
[30]	validation_0-

## Ethereum Fraud

### Autoencoder + Neural DF

#### Original (sigmoid)

In [ ]:
start = datetime.datetime.now()
run_model(45, x_train_eth, x_val_eth, y_train_eth_rl, y_val_eth_rl, 1000, 20)
time_taken = datetime.datetime.now() - start
print(time_taken)

Epoch: 1, Test Accuracy: 0.779375, AUC: 0.714578
Epoch: 2, Test Accuracy: 0.779375, AUC: 0.756671
Epoch: 3, Test Accuracy: 0.779375, AUC: 0.778444
Epoch: 4, Test Accuracy: 0.779375, AUC: 0.736773
Epoch: 5, Test Accuracy: 0.779375, AUC: 0.744141
Epoch: 6, Test Accuracy: 0.779375, AUC: 0.769103
Epoch: 7, Test Accuracy: 0.779375, AUC: 0.781056
Epoch: 8, Test Accuracy: 0.779375, AUC: 0.790193
Epoch: 9, Test Accuracy: 0.779375, AUC: 0.805775
Epoch: 10, Test Accuracy: 0.804063, AUC: 0.820459
Epoch: 11, Test Accuracy: 0.807813, AUC: 0.832553
Epoch: 12, Test Accuracy: 0.770625, AUC: 0.839883
Epoch: 13, Test Accuracy: 0.803125, AUC: 0.844497
Epoch: 14, Test Accuracy: 0.808125, AUC: 0.846074
Epoch: 15, Test Accuracy: 0.791562, AUC: 0.849728
Epoch: 16, Test Accuracy: 0.809688, AUC: 0.852073
Epoch: 17, Test Accuracy: 0.772813, AUC: 0.855094
Epoch: 18, Test Accuracy: 0.808438, AUC: 0.856390
Epoch: 19, Test Accuracy: 0.790000, AUC: 0.861165
Epoch: 20, Test Accuracy: 0.783438, AUC: 0.864191
Epoch: 21

#### ReLU version

In [ ]:
start = datetime.datetime.now()
run_model_relu(45, x_train_eth, x_val_eth, y_train_eth_rl, y_val_eth_rl, 1000, 20)
time_taken = datetime.datetime.now() - start
print(time_taken)

Epoch: 1, Test Accuracy: 0.779375, AUC: 0.804694
Epoch: 2, Test Accuracy: 0.815000, AUC: 0.867953
Epoch: 3, Test Accuracy: 0.815312, AUC: 0.891781
Epoch: 4, Test Accuracy: 0.820625, AUC: 0.902536
Epoch: 5, Test Accuracy: 0.817500, AUC: 0.907120
Epoch: 6, Test Accuracy: 0.834688, AUC: 0.915299
Epoch: 7, Test Accuracy: 0.847500, AUC: 0.919788
Epoch: 8, Test Accuracy: 0.848750, AUC: 0.922853
Epoch: 9, Test Accuracy: 0.849063, AUC: 0.924356
Epoch: 10, Test Accuracy: 0.852187, AUC: 0.927600
Epoch: 11, Test Accuracy: 0.854375, AUC: 0.927807
Epoch: 12, Test Accuracy: 0.885000, AUC: 0.932084
Epoch: 13, Test Accuracy: 0.858437, AUC: 0.928463
Epoch: 14, Test Accuracy: 0.883750, AUC: 0.934856
Epoch: 15, Test Accuracy: 0.880313, AUC: 0.932314
Epoch: 16, Test Accuracy: 0.857812, AUC: 0.934710
Epoch: 17, Test Accuracy: 0.850938, AUC: 0.931651
Epoch: 18, Test Accuracy: 0.887500, AUC: 0.936734
Epoch: 19, Test Accuracy: 0.893437, AUC: 0.936907
Epoch: 20, Test Accuracy: 0.896250, AUC: 0.942115
Epoch: 21

### XGBoost

In [ ]:
start = datetime.datetime.now()
xgbmodel_eth = XGBClassifier(objective='binary:logistic',  n_estimators = 1000, booster = 'gbtree', eval_metric = ['auc'])
xgbmodel_eth.fit(x_train_eth, y_train_eth, eval_set = [(x_val_eth, y_val_eth)], early_stopping_rounds = 10)
time_taken = datetime.datetime.now() - start
print(time_taken)

[0]	validation_0-auc:0.853198
Will train until validation_0-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.853335
[2]	validation_0-auc:0.854387
[3]	validation_0-auc:0.904049
[4]	validation_0-auc:0.904055
[5]	validation_0-auc:0.932984
[6]	validation_0-auc:0.946811
[7]	validation_0-auc:0.942837
[8]	validation_0-auc:0.951333
[9]	validation_0-auc:0.953319
[10]	validation_0-auc:0.958422
[11]	validation_0-auc:0.961091
[12]	validation_0-auc:0.960159
[13]	validation_0-auc:0.961581
[14]	validation_0-auc:0.962737
[15]	validation_0-auc:0.962447
[16]	validation_0-auc:0.963646
[17]	validation_0-auc:0.965628
[18]	validation_0-auc:0.965874
[19]	validation_0-auc:0.965832
[20]	validation_0-auc:0.96535
[21]	validation_0-auc:0.965797
[22]	validation_0-auc:0.966874
[23]	validation_0-auc:0.967594
[24]	validation_0-auc:0.969821
[25]	validation_0-auc:0.970315
[26]	validation_0-auc:0.970392
[27]	validation_0-auc:0.971323
[28]	validation_0-auc:0.9712
[29]	validation_0-auc:0.971984
[30]	validation_0-a